In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")

In [2]:
data_path = "../data_subsidy/"
enroll_df = pd.read_csv(data_path + '2016-Issuer-Enrollment-Disenrollment-Report.csv',na_values=['*'])
enroll_df = enroll_df[['HIOS ID','Policy County FIPS Code','Ever Enrolled Count']].fillna('0')
#clean the column
enroll_df['Ever Enrolled Count'] = enroll_df['Ever Enrolled Count'].apply(lambda x: int(x.replace(',','')) )

# enrollment with issuer characteristics

In [3]:
service_df = pd.read_csv(data_path + 'ServiceArea_PUF_2016.csv', encoding='cp1252')
service_df = service_df[['County','ServiceAreaId','IssuerId']]
service_df = service_df.drop_duplicates()
issuer_df = pd.read_csv('issuer_characteristics.csv')

In [4]:
issuer_service = issuer_df.merge(service_df, how='inner', on=['ServiceAreaId', 'IssuerId'])
issuer_service = issuer_service[ ~issuer_service['County'].isna() ]

#reorder the columns, drop service area
col_order= ['IssuerId','County'] + list(issuer_service.columns)[2:-1]
issuer_service = issuer_service[ col_order ]

#group by county
#pre_cols = list(issuer_service.columns)
issuer_service = issuer_service.groupby(['IssuerId','County'],as_index=False).mean()
#post_cols = list(issuer_service.columns)
print(issuer_service.shape)
#print(len(pre_cols),len(post_cols))

(9413, 71)


In [5]:
enroll_issuer = enroll_df.merge(issuer_service, how='left', right_on=['County', 'IssuerId'],
                                     left_on=['Policy County FIPS Code','HIOS ID'])
enroll_issuer.to_csv('test3.csv')
enroll_issuer = enroll_issuer[~enroll_issuer['IssuerId'].isna()]
print(enroll_issuer.shape)

(5723, 74)


# Merge with county characteristics

In [6]:
county = pd.read_csv('county_characteristics.csv')
result = enroll_issuer.merge(county, how='left', left_on='County', right_on='County')
result =result[~result['FIPS County Code'].isna()]
result.to_csv('merged_characteristics.csv')

# Summary stats

In [7]:
result['Policy County FIPS Code'].nunique()

2141

In [8]:
result['IssuerId'].nunique()

185

In [9]:
col_names = pd.Series(result.columns)
#col_names.to_csv('col_names.csv')

In [10]:
len(list(county.columns))

480